In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
import shutil
from glob import glob
import pysumma as ps
import pysumma.evaluation as pse
from tqdm import tqdm_notebook as tqdm
import xarray as xr
import pandas as pd
from pathlib import Path
import seaborn as sns

sns.set_context('talk')
mpl.style.use('seaborn-bright')
mpl.rcParams['figure.figsize'] = (10, 8)

from glob import glob
import shutil
sites = os.listdir('../sites/')
config = {site: {'file_manager': f'../sites/{site}/file_manager.txt'} for site in sites}
print(len(config))

summa_exe = '/pool0/data/andrbenn/ml_summa/summa/bin/summa.exe'
NPROCS = 4

In [ ]:
def spinup(sim, name, run_option, period, niters, kwargs={}):
    sim.initialize()
    sim.apply_config(kwargs.get('config', {}))
    sim.spinup(run_option, period=period, niters=niters, run_suffix=f'pysumma_spinup_{name}', **kwargs)
    sim.process = None
    return sim

ens = ps.Ensemble(summa_exe, config, num_workers=NPROCS)

In [ ]:
ens.map(spinup, ('local', '365D', 10))

In [ ]:
print(ens.summary()['success'])

In [ ]:
print(ens.summary()['error'])

In [ ]:
for s in sites:
    orig_ic = f'../sites/{s}/params/initial_conditions.nc'
    spin_ic = glob(f'../sites/{s}/params/*restart*.nc')[0]
    shutil.copy2(spin_ic, orig_ic)